# Домашнее задание к занятию «Классификация в АОТ»
# Нетология | 2024г. | Дубовик А.В.|

## Задание: Сделать классификацию данных fakenews

Используя ноутбук занятия (также размещен в папке Materials) и данные fakenews, 

3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn и выше 0.52 для методов на pytorch.

<b>Инструменты и материалы:</b> 
* Использовать ноутбук занятия и материалы на ресурсе: https://github.com/netology-ds-team/nlp-homeworks/tree/main/7_Classification_in_AOT
* sklearn, pytorch

## Решение:

### Импорт библиотек, подготовка данных

#### Импорт библиотек:

In [1]:
import pandas as pd
import numpy as np
from prettytable import PrettyTable

from nltk.tokenize import word_tokenize
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from scipy.stats import uniform


from concurrent.futures import ProcessPoolExecutor

from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
from sklearn.metrics import f1_score

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/alexander/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

В своей работе я старался добавлять как можно больше пояснений, взятых из лекций и используя поиск описаний и грамматических формулировок на разных ресурсах.\
Для себя и для тех, кто будет читать код.

___

#### Загрузка и подготовка данных:

In [3]:
# !wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

In [4]:
df = pd.read_csv('Constraint_Train.csv')


In [5]:
df.head(3)

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake


In [6]:
# создадим список предложений, где каждое предложение представлено в виде списка слов.
sentences = [word_tokenize(text.lower()) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:01<00:00, 5693.37it/s]


Создадим модель Word2Vec. (в лекции была старая версии библиотеки gensim, параметры изменились) \

Параметры:
* sentences: Список предложений, где каждое предложение представлено в виде списка слов. Это основной входной параметр для модели Word2Vec, который определяет, на каких данных будет обучаться модель.
* workers=4: Указывает количество потоков, которые будут использоваться для обучения модели. Это позволяет ускорить процесс обучения, распределяя его по нескольким ядрам процессора.
* vector_size=300: Определяет размерность векторов слов, которые будут обучены моделью. В данном случае, каждое слово будет представлено вектором размерностью 300.
* min_count=3: Этот параметр указывает минимальное количество раз, которое слово должно появиться в корпусе текста, чтобы оно было включено в модель. Слова, встречающиеся реже, будут исключены.
* window=5: Определяет размер окна контекста, в котором слово будет рассматриваться. В данном случае, модель будет учитывать 5 слов слева и справа от текущего слова при обучении векторов слов.
* epochs=15: Указывает количество эпох обучения. Эпоха — это один проход по всему корпусу текста. Большее количество эпох может привести к лучшему обучению модели, но также увеличивает время обучения

In [7]:
%time
model_tweets = Word2Vec(sentences, workers=4, vector_size=300, min_count=3, window=5, epochs=50)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


In [8]:
# нормализации векторов слов в модели Word2Vec (для более ранних версий, в текущей версии этого делать не нужно)
# model_tweets.init_sims()

In [9]:
def get_text_embedding(text):
    '''
    Функция предназначена для получения векторного представления (embedding) текста, используя предварительно обученную модель Word2Vec (model_tweets).
    Полезно для различных задач обработки естественного языка, таких как классификация текста, анализ тональности, рекомендательные системы

    1. Инициализация пустого списка result: Этот список будет использоваться для хранения векторных представлений слов, которые встречаются в тексте.
    2. Токенизация текста и преобразование в нижний регистр: 
        Используя функцию word_tokenize из библиотеки nltk, текст разбивается на отдельные слова (токены), которые затем преобразуются в нижний регистр с помощью метода .lower(). 
        Это делается для обеспечения единообразия текста, так как регистр не влияет на векторное представление слов.
    3. Проверка наличия слова в модели и добавление его вектора в result:
        Для каждого слова в тексте проверяется, есть ли оно в словаре модели Word2Vec (model_tweets.wv). 
        Если слово присутствует, его векторное представление добавляется в список result.
    4. Суммирование векторов слов: 
        Если в тексте было найдено хотя бы одно слово, векторы всех найденных слов суммируются по оси 0 с помощью np.sum(result, axis=0). 
        Это дает векторное представление всего текста, которое является суммой векторов всех слов в тексте.
    5. Возвращение вектора текста или нулевого вектора: 
        Если в тексте не было найдено ни одного слова, возвращается вектор из нулей размерностью 300. 
        В противном случае возвращается суммарный вектор всех слов в тексте.
    '''
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.sum(result, axis=0)
    else:
        result = np.zeros(300)
    return result

In [10]:
# Создадим список векторов текстов, где каждый элемент списка — это векторное представление одного твита
features = [get_text_embedding(text) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:01<00:00, 4395.40it/s]


In [11]:
# Разделим данных на обучающий и тестовый наборы.
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)

### Задание 1. 
3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn

#### 1. Логистическая регрессия

In [12]:
model_lr = LogisticRegressionCV(cv = 5, verbose = 0, n_jobs = -1, scoring = 'roc_auc', solver = 'saga', penalty = 'l1', max_iter = 5000)

In [13]:
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)

In [14]:
# Использование pipeline для объединения масштабирования и подгонки модели.
model_plr = make_pipeline(StandardScaler(), model_lr)

In [15]:
model_plr.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregressioncv',
                 LogisticRegressionCV(cv=5, max_iter=5000, n_jobs=-1,
                                      penalty='l1', scoring='roc_auc',
                                      solver='saga'))])

In [16]:
%time
pred_lr = model_plr.predict(X_test)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [17]:
%time
report_lr = classification_report(pred_lr, y_test, output_dict = True)
print(classification_report(pred_lr, y_test))

CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 6.2 µs
              precision    recall  f1-score   support

        fake       0.92      0.91      0.92      1033
        real       0.92      0.93      0.92      1086

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



In [18]:
# Извлечение f1_score sklearn - LogisticRegressionCV
# f1_score_lr = report_lr['macro avg']['f1-score']
f1_score_lr = report_lr['macro avg']['f1-score']

#### 2. Cлучайный лес с оптимизацией параметров

In [19]:
# Определение параметров для поиска
param_grid = {
    'n_estimators': [100, 200, 300], # Количество деревьев в лесу
    'max_depth': [None, 10, 20, 30], # Максимальная глубина дерева. Установка этого параметра может помочь контролировать переобучение, ограничивая глубину деревьев
    'min_samples_split': [2, 5, 10], # Минимальное количество образцов, необходимых для разделения внутреннего узла
    'min_samples_leaf': [1, 2, 4], # Минимальное количество образцов, необходимых для терминального узла.
    'max_features': ['sqrt'], # Количество признаков, используемых при поиске лучшего разделения.
    'bootstrap': [True, False], # Использование бутстрапа при построении деревьев. Если bootstrap=True, то для каждого дерева выбирается случайная выборка из обучающего набора.
    'criterion': ['gini', 'entropy'], # Функция, используемая для измерения качества разделения. Поддерживаются "gini" для несбалансированной выборки и "entropy" для сбалансированной выборки. 
    'random_state': [42] 
}

In [20]:
model_rf = RandomForestClassifier()

In [21]:
# Инициализация GridSearchCV
grid_search = GridSearchCV(model_rf, param_grid, cv = 5, scoring = 'f1_macro', n_jobs = -1)

In [22]:
# Обучение и поиск оптимальных параметров
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300],
                         'random_state': [42]},
             scoring='f1_macro')

In [23]:
# Вывод лучших параметров
print("Лучшие параметры: ", grid_search.best_params_)


Лучшие параметры:  {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100, 'random_state': 42}


In [24]:
# Предсказание на тестовом наборе
pred_rf = grid_search.predict(X_test)

In [25]:
report_rf = classification_report(pred_rf, y_test, output_dict = True)
print(classification_report(pred_rf, y_test))

              precision    recall  f1-score   support

        fake       0.92      0.93      0.92      1004
        real       0.94      0.92      0.93      1115

    accuracy                           0.93      2119
   macro avg       0.93      0.93      0.93      2119
weighted avg       0.93      0.93      0.93      2119



In [26]:
# Извлечение f1_score sklearn - RandomForestClassifier
f1_score_rf = report_rf['macro avg']['f1-score']

#### 3. Метод опорных векторов (SVM) c оптимизацией гиперпараметров

In [27]:
param_grid_svc = {
        'C': [0.1, 1, 10] , # 100],  # Этот параметр контролирует штраф за ошибки классификации. Более высокие значения C приводят к более сложной модели, где больше признаков выбирается для разделения классов.
        'kernel': ['linear', 'rbf'],
        'random_state': [42] 
}

In [28]:
svc = SVC()

In [29]:
# Определение распределений гиперпараметров
param_distributions = {
    'C': uniform(0.1, 10), # Равномерное распределение между 0.1 и 10
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 3, 50))
}

In [30]:
random_search_cvs = RandomizedSearchCV(estimator = svc, param_distributions = param_distributions, n_iter = 10, cv = 5, n_jobs = -1, verbose = 0)

In [31]:
# grid_search_svc = GridSearchCV(SVC(), param_grid_svc, cv = 5)

In [32]:
random_search_cvs.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f05e49f3490>,
                                        'gamma': ['scale', 'auto', 0.001,
                                                  0.0013257113655901094,
                                                  0.0017575106248547913,
                                                  0.002329951810515372,
                                                  0.0030888435964774815,
                                                  0.004094915062380427,
                                                  0.005428675439323859,
                                                  0.0071968567300115215,
                                                  0.0095409547634...
                                                  0.022229964825261943,
                                                  0.029470517025518096,
                                                  0.03906939937054617,
                                                  0.0517947467923121,
                                                  0.06866488450043001,
                                                  0.09102981779915217,
                                                  0.1206792640639329,
                                                  0.15998587196060574,
                                                  0.21209508879201905,
                                                  0.28117686979742307,
                                                  0.3727593720314938,
                                                  0.49417133613238334,
                                                  0.655128556859551,
                                                  0.868511373751352,
                                                  1.151395399326447,
                                                  1.5264179671752334,
                                                  2.023589647725158, ...],
                                        'kernel': ['linear', 'rbf', 'poly']})

In [33]:
print("Лучшие параметры: ", random_search_cvs.best_params_)

Лучшие параметры:  {'C': 0.4288140461935751, 'gamma': 0.06866488450043001, 'kernel': 'linear'}


In [34]:
# Предсказание на тестовом наборе
pred_svc = random_search_cvs.predict(X_test)

In [35]:
report_svc = classification_report(pred_svc, y_test, output_dict = True)
print(classification_report(pred_svc, y_test))

              precision    recall  f1-score   support

        fake       0.92      0.91      0.92      1028
        real       0.92      0.93      0.92      1091

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



In [36]:
# Извлечение f1_score sklearn - LogisticRegressionCV
f1_score_svc = report_svc['macro avg']['f1-score']

### Задание 2. 
3 раза разными способами получить на задаче классификации значение f1 выше 0.52 для методов на pytorch.

##### RNN - LSTM

In [37]:
labels = (df.label == 'real').astype(int).to_list()

Нужно заранее задать размер для макксимальной длины предложений.

In [38]:
token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(token_lists, key=len))

In [39]:
max_len

1592

In [40]:
from collections import Counter
fd = Counter([len(tokens) for tokens in token_lists])

In [41]:
fd.most_common(10)

[(20, 178),
 (25, 174),
 (22, 170),
 (18, 170),
 (19, 168),
 (21, 168),
 (16, 163),
 (17, 162),
 (15, 160),
 (23, 156)]

Зададим максимум 200.

In [42]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    return result

____

In [43]:
features = [get_word_embedding(text, 200) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:01<00:00, 3926.18it/s]


In [44]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

In [45]:
# Определение устройства
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [46]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(300, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net()
print(net)

Net(
  (lstm): LSTM(300, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [47]:
in_data = torch.tensor(np.array(X_train)).float()
targets = torch.tensor(y_train).float()

In [48]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [49]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [50]:
# def train_model(in_data, targets, batch_size=16, n_epochs=50):
#     for epoch in range(n_epochs):
#         for i in tqdm(range(0, in_data.shape[0], batch_size)):
#             batch_x = in_data[i:i + batch_size]
#             batch_y = targets[i:i + batch_size]
#             optimizer.zero_grad()
#             output = net(batch_x)
#             loss = criterion(output.reshape(-1), batch_y)
#             loss.backward()
#             optimizer.step()
#         print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item()}")

In [51]:
# train_model(in_data, targets)
train_one_epoch(in_data, targets)

100%|██████████| 269/269 [01:23<00:00,  3.21it/s]

tensor(0.6784, grad_fn=<BinaryCrossEntropyBackward0>)


In [52]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

/run/user/1000/app/org.jupyter.JupyterLab/ipykernel_8645/812069076.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  in_data_test = torch.tensor(X_test).float()


In [53]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [54]:
result = (output > 0.5) == targets_test

In [55]:
f1_rnn_lstm = result.sum().item() / len(result)
print(f1_rnn_lstm)

0.5266635205285513


##### BERT модель

In [56]:
# Загрузка предварительно обученного BERT токенизатора и модели
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [57]:
# Токенизация текстов
sentences = []
attention_masks = []
for text in tqdm(df.tweet):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    sentences.append(encoding['input_ids'])
    attention_masks.append(encoding['attention_mask'])


100%|██████████| 6420/6420 [00:04<00:00, 1346.87it/s]


In [58]:
# Преобразование списков в тензоры PyTorch
sentences_tensor = torch.cat(sentences, dim=0)
attention_masks_tensor = torch.cat(attention_masks, dim=0)

labels = (df.label == 'real').astype(int).to_list()
labels_tensor = torch.tensor(labels)


In [59]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(sentences_tensor, labels_tensor, test_size=0.33)

In [60]:
# Подготовка данных для PyTorch
train_dataset = TensorDataset(X_train, y_train, attention_masks_tensor[:len(X_train)])
test_dataset = TensorDataset(X_test, y_test, attention_masks_tensor[len(X_train):])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [61]:
# Определение модели CNN с использованием BERT
class CNN(nn.Module):
    def __init__(self, output_size, bert_model):
        super(CNN, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.5)
        self.label = nn.Linear(768, output_size)
    
    def forward(self, input_sentences, attention_mask):
        with torch.no_grad():
            bert_output = self.bert(input_sentences, attention_mask=attention_mask)[1]
        bert_output = self.dropout(bert_output)
        logits = self.label(bert_output)
        return logits


In [62]:
# Определение устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [63]:
# Определение модели, функции потерь и оптимизатора
model = CNN(output_size=2, bert_model=bert_model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [64]:

# Ранняя остановка
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')

    def __call__(self, val_loss):
        if val_loss < (self.best_loss - self.min_delta):
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

early_stopping = EarlyStopping(patience=5, min_delta=0.01)


In [65]:
# Обучение модели с использованием ранней остановки
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch_idx, (data, target, attention_mask) in enumerate(train_loader):
        data, target, attention_mask = data.to(device), target.to(device), attention_mask.to(device)
        optimizer.zero_grad()
        output = model(data, attention_mask)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    # Валидация
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data, target, attention_mask in test_loader:
            data, target, attention_mask = data.to(device), target.to(device), attention_mask.to(device)
            output = model(data, attention_mask)
            loss = criterion(output, target)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    
    # Ранняя остановка
    if early_stopping(val_loss):
        print(f"Early stopping at epoch {epoch}")
        break


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacity of 3.80 GiB of which 57.50 MiB is free. Process 2309 has 1.04 GiB memory in use. Process 4566 has 530.00 MiB memory in use. Process 5189 has 1.29 GiB memory in use. Process 20713 has 636.00 MiB memory in use. Of the allocated memory 523.02 MiB is allocated by PyTorch, and 44.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Оценка модели
model.eval()
with torch.no_grad():
    y_pred = []
    y_true = []
    for data, target, attention_mask in test_loader:
        data, target, attention_mask = data.to(device), target.to(device), attention_mask.to(device)
        output = model(data, attention_mask)
        _, predicted = torch.max(output, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(target.cpu().numpy())


In [ ]:
f1_bert = f1_score(y_true, y_pred, average='weighted')
print(f"F1 Score: {f1_bert}")

***Коментарий:*** 

пытался использовать ELMA, но не хватило ресурса машины и отказались устанавливаться модули. Убил на это два дня и решил, что на этом хватит.
Хочу услышать про решение, правильно ли все сделал. Так как оптимизировал код как мог, с бертом повозился тоже два дня, но нашел как ускорить процессы.
Проблема была еще с тем что вылетало обучение, так как не хватало GPU у меня всего 4 ГБ. НО в итоге нашел оптимальный вариант.

BERT в итоге отдельным файлом, так как в этом ему всегда не хватает памяти, но f1 score = 0.7798, что считаю не плохим результатом. Название файла bert-2, второй вариант отработал так как сделал кучу оптимизаций по сравнению с первым. Надеюсь так можно сдать домашнее задание, разбив на два файла.

### Вывод 

In [ ]:
# table = PrettyTable()
# table.field_names = ["Модель", "f1_score"]
# table.add_row(["sklearn / LogisticRegressionCV:", f'{f1_score_lr:.2f}'])
# table.add_row(["sklearn / RandomForestClassifier:", f'{f1_score_rf:.2f}'])
# table.add_row(["sklearn / SVC:", f'{f1_score_svc:.2f}'])
# table.add_row(["pytorch / RNN-LSTM:", f'{f1_rnn_lstm:.2f}'])
# # table.add_row(["pytorch / BERT:", f'{f1_bert:.2f}'])
# table.valign = "m"
# print(table.get_string())

Хотел сделать автоматический вывод, но пришлось делать руками, так как все зависло и запускать опять не вижу смысла

sklearn / LogisticRegressionCV: 0.92
sklearn / RandomForestClassifier: 0.93
sklearn / SVC: 0.92
tapytorch / RNN-LSTM: 0.53
pytorch / BERT:0.78
